In [1]:
#====================== Import de librerias =====================#

from pathlib import Path
import json
import gzip
from urllib.request import urlopen
import datetime
import plotly.express as px
import plotly.graph_objects as go
import wget
import logging

import torch
import pandas as pd
import numpy as np
import csv
import os
import scipy.sparse as sp
from typing import Tuple, Dict, Any, List
from tqdm import tqdm, trange
from IPython import embed
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import sys 

current_filename = os.listdir()[3].split(".")[-2][3:]
logfile = "project.log"
old_path = os.getcwd()
os.chdir("..")
execution_path = os.getcwd()

c:\Users\brend\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext tensorboard

logs_base_dir = "runs_"+current_filename
os.environ["run_tensorboard"] = logs_base_dir

os.makedirs(f'{execution_path}/{"4_Modelling"}/{logs_base_dir}', exist_ok=True)
tb_fm = SummaryWriter(log_dir=f'{execution_path}/{"4_Modelling"}/{logs_base_dir}/{logs_base_dir}_FM/')
tb_rnd = SummaryWriter(log_dir=f'{execution_path}/{"4_Modelling"}/{logs_base_dir}/{logs_base_dir}_RANDOM/')

def save_data_configuration(text):
    save_data_dir = "data_config_" + current_filename +  ".txt"
    path = f'{execution_path}/{"4_Modelling"}/{save_data_dir}'
    with open(path, "a") as data_file:
        data_file.write(text+"\n")

    return text

In [3]:
# Let's define some hyper-parameters
hparams = {
    'batch_size':64,
    'num_epochs':12,
    'hidden_size': 32,
    'learning_rate':1e-4,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
#============ Definicion de valores de configuracion ============#

min_reviews, min_usuarios = [6,6]
col_names = {"col_id_reviewer": "reviewerID",
             "col_id_product": "asin",
             "col_unix_time": "unixReviewTime",
             "col_rating": "overall",
             "col_timestamp": "timestamp",
             "col_year": "year"}

csv_filename = execution_path/Path("3_DataPreparation/interactions_minR{}_minU{}.csv".format(min_reviews,min_usuarios))

In [5]:
df = pd.read_csv(csv_filename)
df.head()

,asin,reviewerID,overall,unixReviewTime,timestamp,year
0,0,9132,5.0,1477785600,2016-10-30 02:00:00,1970
1,0,10612,5.0,1467244800,2016-06-30 02:00:00,1970
2,0,257,1.0,1454716800,2016-02-06 01:00:00,1970
3,0,4425,5.0,1434844800,2015-06-21 02:00:00,1970
4,0,2523,4.0,1420329600,2015-01-04 01:00:00,1970


In [6]:
save_data_configuration(str(df.nunique()))
df.nunique()

asin               6178
reviewerID        14138
overall               5
unixReviewTime     3622
timestamp          3622
year                  1
dtype: int64

# Splitting dataset (TLOO strategy)

In [7]:
def split_train_test(data: np.ndarray,
                     n_users: int) -> Tuple[np.ndarray, np.ndarray]:
    # Split and remove timestamp
    train_x, test_x = [], []
    for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
        user_data = data[data[:, 0] == u]
        sorted_data = user_data[user_data[:, -1].argsort()]
        if len(sorted_data) == 1:
            train_x.append(sorted_data[0][:-1])
        else:
            train_x.append(sorted_data[:-1][:, :-1])
            test_x.append(sorted_data[-1][:-1])
    return np.vstack(train_x), np.stack(test_x)

In [8]:
data = df[[*col_names.values()][:3]].astype('int32').to_numpy()
data

array([[      9132,          0, 1477785600],
       [     10612,          0, 1467244800],
       [       257,          0, 1454716800],
       ...,
       [      9051,       6177, 1530144000],
       [      3412,       6177, 1527465600],
       [      9805,       6177, 1527206400]])

In [9]:
add_dims=0
for i in range(data.shape[1] - 1):  # do not affect to timestamp
    # MAKE IT START BY 0
    data[:, i] -= np.min(data[:, i])
    # RE-INDEX
    data[:, i] += add_dims
    add_dims = np.max(data[:, i]) + 1
dims = np.max(data, axis=0) + 1
print("Dim of users: {}\nDim of items: {}\nDims of unixtime: {}".format(dims[0], dims[1], dims[2]))
data

Dim of users: 14138
Dim of items: 20316
Dims of unixtime: 1538006401


array([[      9132,      14138, 1477785600],
       [     10612,      14138, 1467244800],
       [       257,      14138, 1454716800],
       ...,
       [      9051,      20315, 1530144000],
       [      3412,      20315, 1527465600],
       [      9805,      20315, 1527206400]])

In [10]:
train_x, test_x = split_train_test(data, dims[0])
train_x

spliting train/test and removing timestamp...: 100%|██████████| 14138/14138 [00:04<00:00, 2917.18it/s]


array([[    0, 19248],
       [    0, 19249],
       [    0, 14823],
       ...,
       [14137, 14159],
       [14137, 18245],
       [14137, 18904]])

# Negative sampling

In [11]:
train_x = train_x[:, :2]
dims = dims[:2]
print("New dims:",dims)
print("New train_x:\n",train_x)

New dims: [14138 20316]
New train_x:
 [[    0 19248]
 [    0 19249]
 [    0 14823]
 ...
 [14137 14159]
 [14137 18245]
 [14137 18904]]


In [12]:
def build_adj_mx(n_feat:int, data:np.ndarray) -> sp.dok_matrix :
    train_mat = sp.dok_matrix((n_feat, n_feat), dtype=np.float32)
    for x in tqdm(data, desc=f"BUILDING ADJACENCY MATRIX..."):
        train_mat[x[0], x[1]] = 1.0
        train_mat[x[1], x[0]] = 1.0
        # IDEA: We treat features that are not user or item differently because we do not consider
        #  interactions between contexts
        if data.shape[1] > 2:
            for idx in range(len(x[2:])):
                train_mat[x[0], x[2 + idx]] = 1.0
                train_mat[x[1], x[2 + idx]] = 1.0
                train_mat[x[2 + idx], x[0]] = 1.0
                train_mat[x[2 + idx], x[1]] = 1.0
    return train_mat

In [13]:
def ng_sample(data: np.ndarray, dims: list, num_ng:int=4) -> Tuple[np.ndarray, sp.dok_matrix]:
    rating_mat = build_adj_mx(dims[-1], data)
    interactions = []
    min_item, max_item = dims[0], dims[1]
    for num, x in tqdm(enumerate(data), desc='perform negative sampling...'):
        interactions.append(np.append(x, 1))
        for t in range(num_ng):
            j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            # IDEA: Loop to exclude true interactions (set to 1 in adj_train) user - item
            while (x[0], j) in rating_mat or j == int(x[1]):
                j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            interactions.append(np.concatenate([[x[0], j], x[2:], [0]]))
    return np.vstack(interactions), rating_mat

In [14]:
train_x, rating_mat = ng_sample(train_x, dims)
print("Dimensions matrix:\n",dims)
print("\nRating matrix:")
rating_mat

BUILDING ADJACENCY MATRIX...: 100%|██████████| 123226/123226 [00:03<00:00, 40789.19it/s]
perform negative sampling...: 123226it [00:05, 22110.64it/s]


Dimensions matrix:
 [14138 20316]

Rating matrix:


<20316x20316 sparse matrix of type '<class 'numpy.float32'>'
	with 246452 stored elements in Dictionary Of Keys format>

In [15]:
dims[-1]-dims[0]

6178

In [16]:
# Exercise 2

## Evaristo
#### number of ones
print(np.count_nonzero(rating_mat.toarray())/(dims[-1]*dims[-1]))
### number of zeros
print(1 - np.count_nonzero(rating_mat.toarray())/(dims[-1]*dims[-1]))

# ## Brenda
# #### Who sparse is the matrix??
# print(1 - rating_mat.shape[0] / rating_mat.count_nonzero())

0.000597112191656141
0.9994028878083439


In [17]:
train_x[:10]

array([[    0, 19248,     1],
       [    0, 16528,     0],
       [    0, 17066,     0],
       [    0, 17342,     0],
       [    0, 14381,     0],
       [    0, 19249,     1],
       [    0, 18424,     0],
       [    0, 19834,     0],
       [    0, 17237,     0],
       [    0, 19656,     0]])

# Creating dataset class

In [18]:
class PointData(Dataset):
    def __init__(self,
                 data: np.ndarray,
                 dims: list) -> None:
        """
        Dataset formatter adapted point-wise algorithms
        Parameters
        """
        super(PointData, self).__init__()
        self.interactions = data
        self.dims = dims

    def __len__(self) -> int:
        return len(self.interactions)
        
    def __getitem__(self, 
                    index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Return the pairs user-item and the target.
        """
        return self.interactions[index][:-1], self.interactions[index][-1]

train_dataset = PointData(train_x, dims)

In [19]:
train_dataset[0]

(array([    0, 19248]), 1)

# Preparing the test set for inference

In [20]:
test_x

array([[    0, 17249],
       [    1, 18015],
       [    2, 14196],
       ...,
       [14135, 19938],
       [14136, 20214],
       [14137, 15542]])

In [21]:
import math
print(rating_mat.shape)
bits = math.ceil(math.log(rating_mat.shape[0],2))
print("rating_mat contains log2(rating_mat.shape[0]) = {} bits".format(bits))

(20316, 20316)
rating_mat contains log2(rating_mat.shape[0]) = 15 bits


In [22]:
def create_test_no_interactions_stratified_users(train_x: np.ndarray, test_x: np.ndarray, dims_usuarios_productos: Tuple[int, int],  num_samples: int) -> np.ndarray:
    """
    Esta funcion se encarga de crear de manera eficiente un dataset que contenga las interacciones usuario-producto en test que no se hayan producido.
    
    Argumentos:
        train_x (np.ndarray): matriz de entrenamiento con las interacciones usuario-producto previas
        test_x (np.ndarray): matriz de prueba con las interacciones usuario-producto previas
        dims_usuarios_productos (Tuple[int, int]): rango de productos y usuarios disponibles
    
    Retorno:
        np.ndarray: una matriz con todas las interacciones usuario-producto en test que no se hayan producido
    """
    from tqdm import tqdm
    import random
    
    seed = 2   # get same results
    random.seed(a = seed)
    # Identificamos los usuarios presentes en la prueba
    usuarios_test = np.unique(test_x[:, 0])
    # Identificamos el rango de productos disponibles
    total_productos = range(dims_usuarios_productos[0]-1, dims_usuarios_productos[1])
    
    # Recorremos cada usuario presente en la prueba
    for usuario in tqdm(usuarios_test):
        # Identificamos los productos en los que el usuario ha interactuado previamente en entrenamiento
        productos_train = np.unique(train_x[train_x[:, 0] == usuario][:, 1])
        # Seleccionamos al azar 199 productos con los que el usuario no ha interactuado previamente
        productos_a_machear = random.choices(list(set(total_productos) - set(productos_train)), k=num_samples)
        # Creamos una lista de interacciones usuario-producto para este usuario
        # lista_por_usuario = [[usuario, x] for x in productos_a_machear]
        # We must avoid using for loops!
        lista_por_usuario = np.vstack([(np.ones(len(productos_a_machear))*usuario).astype(int), productos_a_machear]).T
        
        # Si es el primer usuario, inicializamos una matriz con sus interacciones
        if usuario == 0:
            zero_positions = lista_por_usuario
        # Si no es el primer usuario, concatenamos sus interacciones a la matriz existente
        else:
            zero_positions = np.concatenate((zero_positions, lista_por_usuario), axis=0)
            
    return zero_positions

In [23]:
# def zero_positions_mode(mode, rating_mat, train_x, test_x, dims):
   
#     if mode == 0:
#         save_data_configuration("\n"+"#"*4+"  zero_positions: all data  "+"#"*4)
#         return np.asarray(np.where(rating_mat.A==0)).T
#     elif mode == 1:
#         zero_true_matrix = np.where(rating_mat.A==0)
#         save_data_configuration("\n"+"#"*4+"  zero_positions: all data separated by rows  "+"#"*4)
#         return np.asarray([zero_true_matrix[0],zero_true_matrix[1]]).T
#     else:
#         save_data_configuration("\n"+"#"*4+"  zero_positions: random sampling  "+"#"*4)
#         zp = create_test_no_interactions(train_x, test_x, dims,  num_samples=199)
#         return zp

In [24]:
# zp = zero_positions_mode(2, rating_mat, train_x, test_x, dims)

In [25]:
# zero_positions = np.asarray(np.where(rating_mat.A==0)).T
save_data_configuration("\n"+"#"*4+"  zero_positions: random sampling  "+"#"*4)
zero_positions = create_test_no_interactions_stratified_users(train_x, test_x, dims,  num_samples=199)
print(save_data_configuration(str(zero_positions.shape)+"\n"))
zero_positions

100%|██████████| 14138/14138 [01:44<00:00, 134.97it/s]

(2813462, 2)



array([[    0, 14448],
       [    0, 20227],
       [    0, 18051],
       ...,
       [14137, 15688],
       [14137, 19466],
       [14137, 18571]])

In [37]:
"""
Esta parte del código se usaba porque el zero positions venia de la rating matrix la cual tenia informacion no útil como (user,user) o (item,item), por eso nos quedábamos con items mayores a dims[0]
Ya que si recordamos, la matrix tenia size: (users + items)
rango items iniciales = 0 ... 6177
rango items actuales  = 14138 ... 20135
"""

# items2compute = []
# for user in trange(dims[0]):
#     aux = zero_positions[zero_positions[:, 0] == user][:, 1]
#     items2compute.append(aux[aux >= dims[0]])
# items2compute[0]

# MUY RÁPIDO !!!!!
items2compute = []
start = 0
num_samples = 199
for user in trange(dims[0]):
    aux = zero_positions[start:start+num_samples,1]
    start += num_samples 
    items2compute.append(aux)
items2compute[2]

100%|██████████| 14138/14138 [02:26<00:00, 96.60it/s] 


array([14448, 20227, 18051, 17816, 15307, 15422, 16102, 19843, 18101,
       15226, 20092, 19105, 18459, 14282, 17910, 19374, 18485, 17528,
       17900, 19059, 14328, 18597, 16772, 15672, 18652, 14259, 15393,
       15491, 17337, 16549, 16356, 14797, 20164, 14885, 18759, 17676,
       16784, 14947, 15412, 16697, 16607, 16740, 16146, 14222, 19641,
       20110, 16271, 17935, 15595, 15488, 18508, 17240, 18610, 15204,
       15575, 14422, 17375, 14937, 17162, 16581, 16410, 16964, 19394,
       16873, 17947, 19523, 14549, 14392, 17538, 17623, 17843, 16529,
       14819, 14413, 16138, 15857, 16418, 16590, 20052, 18460, 18973,
       15309, 16182, 16071, 15238, 14955, 16848, 18547, 17292, 16517,
       17902, 20063, 18220, 19913, 14954, 19590, 14332, 15996, 20197,
       15787, 18294, 14988, 16051, 16552, 20001, 17209, 15479, 18401,
       15681, 17388, 19760, 20221, 14575, 19350, 17819, 15336, 15850,
       17911, 14491, 19673, 18262, 19385, 17472, 18891, 18016, 18251,
       16609, 15891,

In [61]:
def build_test_set(itemsnoninteracted:list, gt_test_interactions: np.ndarray) -> list:
    #max_users, max_items = dims # number users (943), number items (2625)
    test_set = []
    for pair, negatives in tqdm(zip(gt_test_interactions, itemsnoninteracted), desc="BUILDING TEST SET..."):
        # APPEND TEST SETS FOR SINGLE USER
        negatives = np.delete(negatives, np.where(negatives == pair[1]))
        single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
        single_user_test_set[:, 1][1:] = negatives
        test_set.append(single_user_test_set.copy()) # siempre tendremos 1 positivo y el resto negativos
        break
    return test_set

# test_x = build_test_set(items2compute, test_x)
# test_x[0]
build_test_set(items2compute, test_x)

BUILDING TEST SET...: 0it [00:00, ?it/s]

1 [    0 17249]
5 [    0 17249]
6 [[    0 17249]
 [    0 14448]
 [    0 20227]
 [    0 18051]
 [    0 17816]
 [    0 15307]
 [    0 15422]
 [    0 16102]
 [    0 19843]
 [    0 18101]
 [    0 15226]
 [    0 20092]
 [    0 19105]
 [    0 18459]
 [    0 14282]
 [    0 17910]
 [    0 19374]
 [    0 18485]
 [    0 17528]
 [    0 17900]
 [    0 19059]
 [    0 14328]
 [    0 18597]
 [    0 16772]
 [    0 15672]
 [    0 18652]
 [    0 14259]
 [    0 15393]
 [    0 15491]
 [    0 17337]
 [    0 16549]
 [    0 16356]
 [    0 14797]
 [    0 20164]
 [    0 14885]
 [    0 18759]
 [    0 17676]
 [    0 16784]
 [    0 14947]
 [    0 15412]
 [    0 16697]
 [    0 16607]
 [    0 16740]
 [    0 16146]
 [    0 14222]
 [    0 19641]
 [    0 20110]
 [    0 16271]
 [    0 17935]
 [    0 15595]
 [    0 15488]
 [    0 18508]
 [    0 17240]
 [    0 18610]
 [    0 15204]
 [    0 15575]
 [    0 14422]
 [    0 17375]
 [    0 14937]
 [    0 17162]
 [    0 16581]
 [    0 16410]
 [    0 16964]
 [    0 19394]
 [    

[array([[    0, 17249],
        [    0, 14448],
        [    0, 20227],
        [    0, 18051],
        [    0, 17816],
        [    0, 15307],
        [    0, 15422],
        [    0, 16102],
        [    0, 19843],
        [    0, 18101],
        [    0, 15226],
        [    0, 20092],
        [    0, 19105],
        [    0, 18459],
        [    0, 14282],
        [    0, 17910],
        [    0, 19374],
        [    0, 18485],
        [    0, 17528],
        [    0, 17900],
        [    0, 19059],
        [    0, 14328],
        [    0, 18597],
        [    0, 16772],
        [    0, 15672],
        [    0, 18652],
        [    0, 14259],
        [    0, 15393],
        [    0, 15491],
        [    0, 17337],
        [    0, 16549],
        [    0, 16356],
        [    0, 14797],
        [    0, 20164],
        [    0, 14885],
        [    0, 18759],
        [    0, 17676],
        [    0, 16784],
        [    0, 14947],
        [    0, 15412],
        [    0, 16697],
        [    0, 

# Building Factorization Machines model

In [29]:
class FM_operation(torch.nn.Module):

    def __init__(self, 
                 reduce_sum: bool=True) -> None:
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self,
                x: torch.Tensor) -> float:
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        # square_of_sum = np.sum(x, dim=1) ** 2 # ...
        # sum_of_square = np.sum(x ** 2, dim=1) # ...
        
        square_of_sum = torch.pow(torch.sum(x, dim=1),2)
        sum_of_square = torch.sum(torch.pow(x,2), dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix
        

In [30]:
class FactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self, 
                 field_dims: list,
                 embed_dim: float) -> None:
        super().__init__()
        self.linear = torch.nn.Linear(len(field_dims), 1)
        self.embedding = torch.nn.Embedding(field_dims[-1], embed_dim)
        self.fm = FM_operation(reduce_sum=True)

        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, interaction_pairs: torch.Tensor) -> torch.Tensor:
        """
        :param interaction_pairs: Long tensor of size ``(batch_size, num_fields)``
        """
        out = self.linear(interaction_pairs.float()) + self.fm(self.embedding(interaction_pairs))
        return out.squeeze(1)
        
    def predict(self, 
                interactions: np.ndarray,
                device: torch.device) -> torch.Tensor:
        # return the score, inputs are numpy arrays, outputs are tensors
        test_interactions = torch.from_numpy(interactions).to(dtype=torch.long, device=device) #, dtype=torch.long)
        output_scores = self.forward(test_interactions)
        return output_scores

# Pipeline functions

## Training

In [31]:
from statistics import mean

def train_one_epoch(model: torch.nn.Module,
                    optimizer: torch.optim,
                    data_loader: torch.utils.data.DataLoader,
                    criterion: torch.nn.functional,
                    device: torch.device) -> float:
    model.train()
    total_loss = []

    for i, (interactions, targets) in enumerate(data_loader):
        interactions = interactions.to(device)
        targets = targets.to(device)

        predictions = model(interactions)
    
        loss = criterion(predictions, targets.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())

    return mean(total_loss)

# Define metrics

In [32]:
import math

def getHitRatio(recommend_list: list,
                gt_item: int) -> bool:
    if gt_item in recommend_list:
        return 1
    else:
        return 0

def getNDCG(recommend_list: list,
            gt_item: int) -> float:
    idx = np.where(recommend_list == gt_item)[0]
    if len(idx) > 0:
        return math.log(2)/math.log(idx+2)
    else:
        return 0

# Inference


In [33]:
def test(model: torch.nn.Module,
         test_x: np.ndarray,
         device: torch.device,
         topk: int=10) -> Tuple[float, float]:
    # Test the HR and NDCG for the model @topK
    model.eval()

    HR, NDCG = [], []
    for user_test in test_x:
        gt_item = user_test[0][1]
        predictions = model.predict(user_test, device)
        _, indices = torch.topk(predictions, topk)
        recommend_list = user_test[indices.cpu().detach().numpy()][:, 1]

        HR.append(getHitRatio(recommend_list, gt_item))
        NDCG.append(getNDCG(recommend_list, gt_item))
    return mean(HR), mean(NDCG)

# PIPELINE
## Defining the model, the loss and the optimizer



In [34]:
dims = train_dataset.dims
model = FactorizationMachineModel(dims, hparams['hidden_size']).to(device)

criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=hparams['learning_rate'])

## Random evaluation

In [35]:
import random
class RandomModel(torch.nn.Module):
    def __init__(self, 
                 dims: list) -> None:
        super(RandomModel, self).__init__()
        """
        Simple random based recommender system
        """
        self.all_items = list(range(dims[0], dims[1]))

    def forward(self) -> None:
        pass

    def predict(self,
                interactions: np.ndarray,
                device=None) -> torch.Tensor:
        return torch.FloatTensor(random.sample(self.all_items, len(interactions)))

rnd_model = RandomModel(dims)

## Final pipeline

In [36]:
data_loader = DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True, num_workers=0)

# Start training the model

In [38]:
# DO EPOCHS NOW
topk = 10
for epoch_i in range(hparams['num_epochs']):
    #data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(model, optimizer, data_loader, criterion, device)
    hr, ndcg = test(model, test_x, device, topk=topk)

    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
    print('\n')
 
    tb_fm.add_scalar('train/loss', train_loss, epoch_i)
    tb_fm.add_scalar('eval/HR@{topk}', hr, epoch_i)
    tb_fm.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)

    hr, ndcg = test(rnd_model, test_x, device, topk=topk)
    tb_rnd.add_scalar('eval/HR@{topk}', hr, epoch_i)
    tb_rnd.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)

epoch 0:
training loss = 309.8556 | Eval: HR@10 = 0.0395, NDCG@10 = 0.0181 


epoch 1:
training loss = 0.5132 | Eval: HR@10 = 0.1502, NDCG@10 = 0.0949 


epoch 2:
training loss = 0.5054 | Eval: HR@10 = 0.1086, NDCG@10 = 0.0683 


epoch 3:
training loss = 0.4907 | Eval: HR@10 = 0.1318, NDCG@10 = 0.0809 


epoch 4:
training loss = 0.4683 | Eval: HR@10 = 0.1715, NDCG@10 = 0.1036 


epoch 5:
training loss = 0.4477 | Eval: HR@10 = 0.1937, NDCG@10 = 0.1162 


epoch 6:
training loss = 0.4291 | Eval: HR@10 = 0.2109, NDCG@10 = 0.1257 


epoch 7:
training loss = 0.4143 | Eval: HR@10 = 0.2266, NDCG@10 = 0.1335 


epoch 8:
training loss = 0.4038 | Eval: HR@10 = 0.2379, NDCG@10 = 0.1396 


epoch 9:
training loss = 0.3973 | Eval: HR@10 = 0.2465, NDCG@10 = 0.1442 


epoch 10:
training loss = 0.3923 | Eval: HR@10 = 0.2525, NDCG@10 = 0.1478 


epoch 11:
training loss = 0.3867 | Eval: HR@10 = 0.2494, NDCG@10 = 0.1461 




# Visualization

In [42]:
os.chdir(os.getcwd() / Path("4_Modelling"))

"kill" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [ ]:
%tensorboard --logdir run_tensorboard